***GENERATED CODE FOR BikeRentalEstimationApp PIPELINE***

**CONNECTOR FUNCTIONS TO READ DATA FROM DATABRICKS FILESYSTEM**

In [ ]:
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class DBFSConnector:

    def fetch(inStages, inStagesData, stageId, spark, config):
        df = spark.read.\
            options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                    inferschema='true',
                    delimiter=eval(config)["delimiter"])\
            .csv(eval(config)['url'])
        display(df.limit(2).toPandas())
        return df

    def put(inStages, inStagesData, stageId, spark, config):
        return inStagesData.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                                        delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


**TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA**

In [ ]:
from pyspark.sql.functions import col, dayofmonth, month, year
import json
from pyspark.sql.functions import col, when
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def replaceByMean(self, feature, df, mean_=-1):

        meanValue = df.select(mean(col(feature.name)).alias(
            'mean')).collect()[0]["mean"]
        df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                                         meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        maxValue = df.select(max(col(feature.name)).alias('max')).collect()[
            0]["max"]
        df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        minValue = df.select(min(col(feature.name)).alias('min')).collect()[
            0]["min"]
        df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        stddevValue = df.select(stddev(col(feature.name)).alias(
            'stddev')).collect()[0]["stddev"]
        df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        fillValue = df.where(col(feature.name).isNotNull()
                             ).head(1)[0][feature.name]
        df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def ExtractDateTransform(df, params):
    print("ExtractDate:ExtractDateTransform Started parameters : \n",
          params, "\n", df.show())
    transform_params = params
    dfReturn = df
    feature = transform_params['feature']
    dfReturn = dfReturn.fillna({feature: ''})
    # dfReturn = dfReturn.withColumn(feature+'_weekofyear', weekofyear(col(feature)))
    # dfReturn = dfReturn.withColumn(feature+'dayofyear', dayofyear(col(feature)))
    dfReturn = dfReturn.withColumn(
        feature+'dayofmonth', dayofmonth(col(feature)))
    # dfReturn = dfReturn.withColumn(feature+'dayofweek', dayofweek(col(feature)))
    dfReturn = dfReturn.withColumn(feature+'month', month(col(feature)))
    dfReturn = dfReturn.withColumn(feature+'year', year(col(feature)))
    dfReturn = dfReturn.drop(feature)
    # print(dfReturn.printSchema())
    print("ExtractDate:ExtractDateTransform Output Dataframe :", dfReturn.head())
    return dfReturn


Feature_Transformations_Methods = {
    "Extract Date": ExtractDateTransform,

}


class TransformationMain:
    # TODO: change df argument in run with following
    def run(inStages, inStagesData, stageId, spark, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = inStagesData[inStages[0]]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        for transformation in featureData:
            feature = transformation["feature"]
            if transformation["transformation"] != '' and transformation["selected"].lower() == "true" and not (feature.__contains__("_transform")):
                transformationDF = Feature_Transformations_Methods["%s" % transformation["transformation"]](
                    transformationDF, transformation)

        display(transformationDF.limit(2).toPandas())
        return transformationDF


**AUTOML FUNCTIONS**

In [ ]:
from sklearn.model_selection import train_test_split
from tpot import TPOTRegressor
import pyspark


def functionRegression(sparkDF, listOfFeatures, label):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = sparkDF.toPandas()
    df.columns.intersection(listOfFeatures)
    X = df.drop(label, axis=1).values
    y = df[label].values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=1, test_size=0.1)
    tpotModel = TPOTRegressor(verbosity=3, generations=10, max_time_mins=15,
                              n_jobs=-1, random_state=25, population_size=15)
    tpotModel.fit(X_train, y_train)
    display(" Error rate of Model : %s" % tpotModel.score(X_test, y_test))


**READING DATAFRAME**

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

try: 
	BikeRentalEstimation_App_DBFS = DBFSConnector.fetch([], {}, "5ea80fb261f8a37af50e1c2c", spark, "{'url': '/Demo/Demand_Estimate_ Bike_Rental.csv', 'file_type': 'Delimeted', 'delimiter': ',', 'is_header': 'Use Header Line'}")

except Exception as ex: 
	logging.error(ex)


**TRANSFORMING DATAFRAME**

In [ ]:
try: 
	BikeRentalEstimation_App_AutoFE = TransformationMain.run(["5ea80fb261f8a37af50e1c2c"],{"5ea80fb261f8a37af50e1c2c": BikeRentalEstimation_App_DBFS}, "5ea80fb261f8a37af50e1c2d", spark,json.dumps( {"FE": [{"feature": "instant", "transformation": "", "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "12197", "mean": "8680.57", "stddev": "5028.01", "min": "1", "max": "17378", "missing": "0"}}, {"transformationsData": {"feature_label": "dteday"}, "feature": "dteday", "type": "date", "selected": "True", "replaceby": "random", "stats": {"count": "", "mean": "", "stddev": "", "min": "", "max": "", "missing": "0"}, "transformation": "Extract Date"}, {"feature": "season", "transformation": "", "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "12197", "mean": "2.5", "stddev": "1.11", "min": "1", "max": "4", "missing": "0"}}, {"feature": "yr", "transformation": "", "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "12197", "mean": "0.5", "stddev": "0.5", "min": "0", "max": "1", "missing": "0"}}, {"feature": "mnth", "transformation": "", "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "12197", "mean": "6.54", "stddev": "3.44", "min": "1", "max": "12", "missing": "0"}}, {"feature": "hr", "transformation": "", "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "12197", "mean": "11.6", "stddev": "6.92", "min": "0", "max": "23", "missing": "0"}}, {"feature": "holiday", "transformation": "", "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "12197", "mean": "0.03", "stddev": "0.17", "min": "0", "max": "1", "missing": "0"}}, {"feature": "weekday", "transformation": "", "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "12197", "mean": "3.0", "stddev": "2.0", "min": "0", "max": "6", "missing": "0"}}, {"feature": "workingday", "transformation": "", "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "12197", "mean": "0.68", "stddev": "0.47", "min": "0", "max": "1", "missing": "0"}}, {"feature": "weathersit", "transformation": "", "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "12197", "mean": "1.42", "stddev": "0.64", "min": "1", "max": "4", "missing": "0"}}, {"feature": "temp", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "12197", "mean": "0.5", "stddev": "0.19", "min": "0.02", "max": "0.98", "missing": "0"}, "transformation": ""}, {"feature": "atemp", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "12197", "mean": "0.48", "stddev": "0.17", "min": "0.0", "max": "1.0", "missing": "0"}, "transformation": ""}, {"feature": "hum", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "12197", "mean": "0.63", "stddev": "0.19", "min": "0.0", "max": "1.0", "missing": "0"}, "transformation": ""}, {"feature": "windspeed", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "12197", "mean": "0.19", "stddev": "0.12", "min": "0.0", "max": "0.8507", "missing": "0"}, "transformation": ""}, {"feature": "casual", "transformation": "", "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "12197", "mean": "35.46", "stddev": "48.97", "min": "0", "max": "362", "missing": "0"}}, {"feature": "registered", "transformation": "", "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "12197", "mean": "153.2", "stddev": "150.26", "min": "0", "max": "885", "missing": "0"}}, {"feature": "cnt", "transformation": "", "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "12197", "mean": "188.66", "stddev": "180.19", "min": "1", "max": "976", "missing": "0"}}]}))

except Exception as ex: 
	logging.error(ex)


**TRAIN MODEL**

In [ ]:
try: 
	functionRegression(BikeRentalEstimation_App_AutoFE, ["instant", "dteday", "season", "yr", "mnth", "hr", "holiday", "weekday", "workingday", "weathersit", "temp", "atemp", "hum", "windspeed", "casual", "registered"], "cnt")

except Exception as ex: 
	logging.error(ex)
